# Birder - Getting Started

In this notebook we will explore some of the API's provided by Birder.

Before we start, if you're running in Colab, make sure to install Birder first.
Numpy 2.0 and above are not yet supported on Colab, so you might have to downgrade as well.

In [ ]:
# When running in Colab
# !pip install birder
# !pip install "numpy<2.0"

# When running in a cloned repository (instead of pip installation)
# %cd ..

In [ ]:
import birder
import torch
import torch.nn.functional as F
from birder.core.inference.classification import infer_batch
from birder.core.results.gui import show_top_k
from PIL import Image

In [ ]:
birder.__version__

## Exploring Models

We can list all pretrained models according to any filter (or without).

Let's look at all models that were trained on the il-common dataset and load one of them.

In [ ]:
birder.list_pretrained_models("*il-common*")

In [ ]:
(net, class_to_idx, signature, rgb_values) = birder.load_pretrained_model("xcit_nano16_il-common", inference=True)

# Get the image size the model was trained on
size = birder.get_size_from_signature(signature)

# Create an inference transform
transform = birder.classification_transform(size, rgb_values)

## Inference

Now we shall fetch an example image (of a Eurasian teal) and try to classify it.

In [ ]:
# In case it's a first run, create the data dir
# !mkdir data

In [ ]:
image_path = "data/example.jpeg"
birder.common.cli.download_file("https://f000.backblazeb2.com/file/birder/data/img_001.jpeg", image_path)

In [ ]:
img = Image.open(image_path)
input_tensor = transform(img).unsqueeze(dim=0)
(out, _) = infer_batch(net, input_tensor)
show_top_k(image_path, out.squeeze(), class_to_idx, "Eurasian teal")

In [ ]:
# The model failed to classify it correctly, this is probably because the object is small
# and we are using a low resolution compact model.
#
# We will try again using an aggresive center crop.
transform = birder.classification_transform(size, rgb_values, center_crop=0.5)
input_tensor = transform(img).unsqueeze(dim=0)
(out, _) = infer_batch(net, input_tensor)
show_top_k(image_path, out.squeeze(), class_to_idx, "Eurasian teal")

## Fine-tuning

We shall now fine-tune the model on an example dataset.

For this example we will use the Caltech-UCSD Birds-200-2011 dataset - <https://authors.library.caltech.edu/records/cvm3y-5hh21>.

It has about ~12K images of 200 species.

We will first do simple linear probing and later a full training.

In [ ]:
from birder.datahub.classification import CUB_200_2011
from birder.scripts import train

In [ ]:
CUB_200_2011(download=True)

In [ ]:
# To prepare the model for training we must give it an initial epoch number
!mv models/xcit_nano16_il-common.pt models/xcit_nano16_il-common_0.pt

In [ ]:
# Linear probing
# The reset head argument will fit a new classification head and freeze the rest of the network
args = train.args_from_dict(
    network="xcit_nano16",
    tag="il-common",
    num_workers=2,
    lr=0.1,
    lr_scheduler="cosine",
    epochs=5,
    resume_epoch=0,
    reset_head=True,
    size=256,
    data_path="data/CUB_200_2011/training",
    val_path="data/CUB_200_2011/validation",
)

In [ ]:
train.train(args)

In [ ]:
# Full fine-tuning for 10 epochs
args = train.args_from_dict(
    network="xcit_nano16",
    tag="il-common",
    num_workers=2,
    opt="adamw",
    lr=0.0001,
    lr_scheduler="cosine",
    lr_cosine_min=1e-7,
    epochs=15,
    resume_epoch=5,
    size=256,
    wd=0.05,
    norm_wd=0,
    grad_accum_steps=2,
    smoothing_alpha=0.1,
    mixup_alpha=0.2,
    cutmix=True,
    aug_level=4,
    clip_grad_norm=1,
    fast_matmul=True,
    # compile=True,
    data_path="data/CUB_200_2011/training",
    val_path="data/CUB_200_2011/validation",
)

In [ ]:
train.train(args)

## Evaluate

In [ ]:
from birder.common.fs_ops import load_model
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda")
(net, class_to_idx, signature, rgb_values) = load_model(
    device, "xcit_nano16", tag="il-common", epoch=15, inference=True
)

In [ ]:
dataset = CUB_200_2011(split="validation", transform=transform)
inference_loader = DataLoader(
    dataset,
    batch_size=128,
    num_workers=2,
)

In [ ]:
results = birder.evaluate_classification(device, net, inference_loader, class_to_idx)

In [ ]:
results.log_short_report()

In [ ]:
results.most_confused()